# Processing a HCP Dataset

Here we have run an HCP dataset through DSI Studio using the recommended parameters from the documentation. This includes

  * Gradient unwarping
  * Motion/Eddy correction
  * TopUp
  
The images (dwi mask, dwi data and graddev files) were downloaded directly from connectomedb.org. The resulting fib file is in 101915/data/input.

### Loading DSI Studio files

Here a ``MITTENS`` object is created using default parameters.

In [1]:
from mittens import MITTENS
mitn = MITTENS(fibgz_file="101915/data/input/101915.src.gz.odf8.f5rec.fy.gqi.1.25.fib.gz")

INFO:mittens._mittens:
Using
------
  Step Size:		0.8660 Voxels 
  ODF Resolution:	odf8
  Max Angle:		35.00 Degrees
  Angle Weights:	flat
  Angle weight power:	1.0
INFO:mittens._mittens:Loading DSI Studio fib file
INFO:mittens._mittens:Loading 101915/data/input/101915.src.gz.odf8.f5rec.fy.gqi.1.25.fib.gz
INFO:mittens._mittens:Loaded 101915/data/input/101915.src.gz.odf8.f5rec.fy.gqi.1.25.fib.gz
INFO:mittens._mittens:Loading ODF data
INFO:mittens._mittens:Loaded ODF data: (750138, 321)


### Calculating transition probabilities

We see in the output above that the default parameters of $\theta_{max}=35$ degrees and $s=\sqrt{3}/2$ voxels will be used for calculating transition probabilities. Now we actually calculate the probabilities and save them in NIfTI format.

In [2]:
import os
#os.makedirs("101915/data/mittens_output")
mitn.calculate_transition_probabilities(output_prefix="HCP")

INFO:mittens._mittens:ODF 0/750138
INFO:mittens._mittens:ODF 10000/750138
INFO:mittens._mittens:ODF 20000/750138
INFO:mittens._mittens:ODF 30000/750138
INFO:mittens._mittens:ODF 40000/750138
INFO:mittens._mittens:ODF 50000/750138
INFO:mittens._mittens:ODF 60000/750138
INFO:mittens._mittens:ODF 70000/750138
INFO:mittens._mittens:ODF 80000/750138
INFO:mittens._mittens:ODF 90000/750138
INFO:mittens._mittens:ODF 100000/750138
INFO:mittens._mittens:ODF 110000/750138
INFO:mittens._mittens:ODF 120000/750138
INFO:mittens._mittens:ODF 130000/750138
INFO:mittens._mittens:ODF 140000/750138
INFO:mittens._mittens:ODF 150000/750138
INFO:mittens._mittens:ODF 160000/750138
INFO:mittens._mittens:ODF 170000/750138
INFO:mittens._mittens:ODF 180000/750138
INFO:mittens._mittens:ODF 190000/750138
INFO:mittens._mittens:ODF 200000/750138
INFO:mittens._mittens:ODF 210000/750138
INFO:mittens._mittens:ODF 220000/750138
INFO:mittens._mittens:ODF 230000/750138
INFO:mittens._mittens:ODF 240000/750138
INFO:mittens._

This writes out NIfTI images for the transition probabilities to each neighbor using both the singleODF and doubleODF methods. Additionally, you will find volumes written out for CoDI and CoAsy. Writing to NIfTI is useful to quickly assess the quality of the output. the CoDI volumes should look a lot like GFA or FA images.

One can load theses images directly to create a ``MITTENS`` object. This bypasses the need to re-calculate transition probabilities and is the recommended way to access transition probabilities before building and saving Voxel Graphs. 

### Loading from NIfTI outputs

Here we demonstrate loading transition probabilites from NIfTI files. We verify that their contents are identical to those generated by calculating transition probabilities from a ``fib.gz`` file.



In [4]:
nifti_mitn = MITTENS(nifti_prefix="HCP")

import numpy as np
print("singleODF outputs are identical:", 
          np.allclose(mitn.singleODF_results,nifti_mitn.singleODF_results))
print("doubleODF outputs are identical:", 
          np.allclose(mitn.doubleODF_results,nifti_mitn.doubleODF_results,equal_nan=True))

INFO:mittens._mittens:
Using
------
  Step Size:		0.8660 Voxels 
  ODF Resolution:	odf8
  Max Angle:		35.00 Degrees
  Angle Weights:	flat
  Angle weight power:	1.0
INFO:mittens._mittens:Loading output from pre-existing NIfTIs
INFO:mittens.spatial:Loading NIfTI Image HCP_mask.nii.gz
INFO:mittens._mittens:Used  to mask from 3658350 to 750138 voxels
INFO:mittens._mittens:Loading singleODF results
INFO:mittens._mittens:Loading NIfTI Image HCP_singleODF_a_prob.nii.gz
INFO:mittens.spatial:Loading NIfTI Image HCP_singleODF_a_prob.nii.gz
INFO:mittens._mittens:Loading NIfTI Image HCP_singleODF_ai_prob.nii.gz
INFO:mittens.spatial:Loading NIfTI Image HCP_singleODF_ai_prob.nii.gz
INFO:mittens._mittens:Loading NIfTI Image HCP_singleODF_as_prob.nii.gz
INFO:mittens.spatial:Loading NIfTI Image HCP_singleODF_as_prob.nii.gz
INFO:mittens._mittens:Loading NIfTI Image HCP_singleODF_i_prob.nii.gz
INFO:mittens.spatial:Loading NIfTI Image HCP_singleODF_i_prob.nii.gz
INFO:mittens._mittens:Loading NIfTI Image H

singleODF outputs are identical: True
doubleODF outputs are identical: True


You'll notice loading from NIfTI is much faster!

## Building and saving a voxel graph

Transition probabilities need to be converted to edge weights somehow. The ``MITTENS`` object accomplishes this through the ``build_graph`` function, which offers a number of options. Here is the relevant portion of the ``build_graph`` documentation

    Schemes for shortest paths:
    ---------------------------

    ``"negative_log_p"``:
      Transition probabilities are log transformed and made negative.  This is similar to the Zalesky 2009 strategy.

    ``"minus_iso_negative_log"``:
      Isotropic probabilities are subtracted from transition probabilities. Edges are not added when transition probabilities are less than the isotropic probability.

    ``"minus_iso_scaled_negative_log"``:
      Same as ``"minus_iso_negative_log"`` except probabilities are re-scaled to sum to 1 *before* the log transform is applied. 
      
      
You also have to pick whether to use singleODF or doubleODF probabilities. You can easily create graphs using either.

In [ ]:
doubleODF_nlp = mitn.build_graph(doubleODF=True, weighting_scheme="negative_log_p")
doubleODF_nlp.save("fcup_doubleODF_nlp.mat")

### Notes

#### Masking
If no spatial mask is specified when the ``MITTENS`` object is constructed, the generous brain mask estimated by DSI Studio will be used.  This means the file sizes will be somewhat larger and the calculations will take longer. However, we recommend using the larger mask, as you can easily apply masks to the voxel graphs.  It's better to have a voxel and not need it than to re-run transition probability calculation.

#### Affines
``MITTENS`` mimics DSI Studio. Internally everything is done in LPS+ orientation and results are written out in RAS+ orientation. The affine used in the transition probability NIfTI images is the same as the images written by DSI Studio. This is a bizarre area of scanner coordinates and more closely matches the coordinates used by streamlines.

### Verifying that the Voxel Graph is preserved

Here we load a Voxel Graph directly from a matfile and check that it exactly matches the graph produced above.

In [ ]:
from mittens import VoxelGraph
mat_vox_graph = VoxelGraph("101915/data/mittens_output/HCP_doubleODF_nlp.mat")


from networkit.algebraic import adjacencyMatrix
matfile_adj = adjacencyMatrix(mat_vox_graph.graph,matrixType="sparse")
mitn_adj = adjacencyMatrix(doubleODF_nlp.graph,matrixType="sparse")


mat_indices = np.nonzero(matfile_adj)
indices = np.nonzero(mitn_adj)

print("row indices match:", np.all(mat_indices[0] == indices[0]))
print("column indices match:",np.all(mat_indices[1] == indices[1]))

print("Weights all match:", np.allclose(matfile_adj[indices],mitn_adj[indices]))

### Conclusions

Here we've shown that transition probabilities and edge weights are preserved over the three ways they can be represented in ``MITTENS``. Next we show useful operations that can be performed using a VoxelGraph.